<a href="https://colab.research.google.com/github/MJLee25/app_trial/blob/main/%5B%EC%A0%9C%EC%B6%9C%EC%9A%A9%5D%EC%96%B4%ED%94%8C%EB%A6%AC%EC%BC%80%EC%9D%B4%EC%85%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import geopandas as gpd
import shapely.geometry
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from shapely import wkt
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely.geometry import Point, Polygon

In [30]:
# 드라이브와 연결
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
unscaled_train = pd.read_csv("/content/drive/MyDrive/빅콘_다시/데이터/차원축소/[vif_result추가데이터]unscaled_train.csv")
unscaled_train["EMD_CD"] = unscaled_train["EMD_CD"].astype(str)
unscaled_train["key"] = unscaled_train["key"].astype(str)
unscaled_train["dealt_yr"] = unscaled_train['dealt_yr'].astype(str)
unscaled_train

,gfa_dlt,land_area,floor_gr,floor_bm,year_con,교통_도보접근시간,학교_차량접근시간,교통_차량접근시간,학교_대중교통접근시간,병원_대중교통접근시간,...,add_dong,geometry,bld_type,EMD_CD,key,price_tr,의료,성비,총인구,GRDP
0,27202.05,3602.0,12.0,4.0,1985.0,3.917742,0.823476,17.158817,10.572581,12.580645,...,여의도동,POLYGON ((948901.6666324371 1949279.7103949625...,집합,11560110,1,14600,365785,0.500753,382352,28194672
1,3516.43,713.6,7.0,2.0,2004.0,4.275025,1.127415,21.029943,13.713139,10.803410,...,논현동,POLYGON ((959462.1692613533 1946535.4616427755...,일반,11680108,2,8100,540623,0.479606,578114,57487436
2,25984.49,2705.0,17.0,4.0,1995.0,4.102086,0.316556,18.921771,8.910538,10.889133,...,충정로3가,POLYGON ((952385.8893198308 1951907.4372649118...,일반,11410102,3,13900,285266,0.487596,310376,7195459
3,31894.78,3772.0,17.0,4.0,1987.0,4.102086,0.316556,18.921771,8.910538,10.889133,...,충정로3가,POLYGON ((952385.8893198308 1951907.4372649118...,집합,11410102,4,10500,285266,0.487596,310376,7195459
4,4370.31,730.5,5.0,4.0,2012.0,4.704024,1.726482,19.674376,13.085201,10.418577,...,한남동,POLYGON ((956238.2713397081 1950166.6098483382...,일반,11170131,5,25200,213789,0.485601,235951,9686230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,87901.58,5929.7,15.0,7.0,2018.0,8.278289,0.908917,25.188476,10.510441,17.764888,...,백현동,"POLYGON ((964616.7413705747 1931702.376122126,...",일반,41135110,940,26000,1325065886,0.494850,919747,50302710
940,134124.39,24778.7,5.0,4.0,2000.0,7.164773,0.833810,28.615593,10.601326,12.844066,...,구미동,"POLYGON ((964736.0567730014 1927185.795389603,...",집합,41135114,941,12300,1325065886,0.494850,919747,50302710
941,197236.69,13352.1,15.0,7.0,2021.0,8.278289,0.908917,25.188476,10.510441,17.764888,...,백현동,"POLYGON ((964616.7413705747 1931702.376122126,...",일반,41135110,942,26200,1325065886,0.494850,919747,50302710
942,17439.35,2243.8,8.0,4.0,1998.0,9.313344,0.522833,25.624317,9.806801,15.978877,...,서현동,"POLYGON ((966038.3705994984 1932053.418835807,...",일반,41135105,943,17900,1325065886,0.494850,919747,50302710


In [32]:
X_matrix = unscaled_train.drop(["add_si", "add_gu", 'add_dong', "EMD_CD", "geometry", "key", "price_tr"], axis=1)
X_matrix.columns

Index(['gfa_dlt', 'land_area', 'floor_gr', 'floor_bm', 'year_con', '교통_도보접근시간',
       '학교_차량접근시간', '교통_차량접근시간', '학교_대중교통접근시간', '병원_대중교통접근시간', '학교_도보접근수',
       '병원_도보접근수', '교통_도보접근수', '학교_차량접근수', '교통_차량접근수', '교통_대중교통접근수',
       'avg_부동산시장_소비심리지수', '차주가중종합지수_attitude', '대기업_attitude',
       '상호저축은행_attitude', '신용카드회사_attitude', '생명보험회사_attitude',
       '차주가중종합지수_risk', '대기업_risk', '상호저축은행_risk', '신용카드회사_risk',
       '차주가중종합지수_demand', '대기업_demand', '상호저축은행_demand', '신용카드회사_demand',
       '상호금융조합_demand', '생명보험회사_demand', 'avg_경제심리지수', '상권업종', 'M2평균', '증가율평균',
       'dealt_yr', 'dealt_qr', 'bld_type', '의료', '성비', '총인구', 'GRDP'],
      dtype='object')

In [33]:
Y_vector = unscaled_train["price_tr"]
Y_vector.head()

,price_tr
0,14600
1,8100
2,13900
3,10500
4,25200


In [34]:
## Encoding Categorical --> 0,1,2 수치화
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

X_matrix['dealt_qr'] = le.fit_transform(X_matrix['dealt_qr'])
X_matrix['dealt_yr'] = le.fit_transform(X_matrix['dealt_yr'])
X_matrix['bld_type'] = le.fit_transform(X_matrix['bld_type'])
X_matrix[["dealt_qr", "dealt_yr", "bld_type"]].head(10)

,dealt_qr,dealt_yr,bld_type
0,0,0,1
1,0,0,0
2,0,0,0
3,0,0,1
4,0,0,0
5,0,0,0
6,0,0,1
7,0,0,0
8,0,0,0
9,0,0,0


In [35]:
unscaled_test = pd.read_csv("/content/drive/MyDrive/빅콘_다시/데이터/차원축소/[vif_result추가데이터]unscaled_test.csv")
unscaled_test["EMD_CD"] = unscaled_test["EMD_CD"].astype(str)
unscaled_test["dealt_yr"] = unscaled_test['dealt_yr'].astype(str)
unscaled_test.head()

,gfa_dlt,land_area,floor_gr,floor_bm,year_con,교통_도보접근시간,학교_차량접근시간,교통_차량접근시간,학교_대중교통접근시간,병원_대중교통접근시간,...,add_gu,add_dong,geometry,bld_type,EMD_CD,price_tr,의료,성비,총인구,GRDP
0,4518.11,624.50,13,3,1989,2.214286,1.604762,19.433333,12.083333,10.238095,...,종로구,관훈동,POLYGON ((954463.3552837974 1953084.3235795044...,일반,11110128,18600,298391378,0.482768,139417,35719537
1,27897.50,3081.10,18,4,1985,4.296371,1.034872,16.944489,10.997984,12.506720,...,중구,남대문로5가,POLYGON ((953683.5529665507 1951253.6670277438...,일반,11140118,36700,256515340,0.483538,121312,61342710
2,41139.63,3964.89,20,2,1970,4.296371,1.034872,16.944489,10.997984,12.506720,...,중구,남대문로5가,POLYGON ((953683.5529665507 1951253.6670277438...,일반,11140118,33400,256515340,0.483538,121312,61342710
3,41676.87,2846.80,20,7,1997,2.795082,1.800000,18.066667,13.504098,10.969945,...,중구,다동,POLYGON ((954306.9581746012 1952309.5642655282...,집합,11140102,31000,256515340,0.483538,121312,61342710
4,4307.51,803.80,8,2,2003,5.195528,0.287018,20.047703,11.541862,13.250130,...,강남구,청담동,POLYGON ((959934.7556779666 1948388.9741884433...,일반,11680104,31500,1861652875,0.478130,544873,77924019


In [36]:
X_test = unscaled_test.drop(["add_si", "add_gu", 'add_dong', "EMD_CD", "geometry", "price_tr"], axis=1)
X_test.columns

X_test['dealt_qr'] = le.fit_transform(X_test['dealt_qr'])
X_test['dealt_yr'] = le.fit_transform(X_test['dealt_yr'])
X_test['bld_type'] = le.fit_transform(X_test['bld_type'])

Y_test = unscaled_test["price_tr"]
Y_test.head()

,price_tr
0,18600
1,36700
2,33400
3,31000
4,31500


## RF

In [37]:
n_estimators = [5,10,20,40,50,80,100]    #트리의 수
max_features = ['auto', 'sqrt']  #나누는 수(트리의 무작위수)
max_depth = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120]   # 트리의 최대 깊이
min_samples_split = [2, 4, 6, 8, 10]
min_samples_leaf = [1, 2, 3, 4]
bootstrap = [True, False] # method used to sample data points

random_grid = {'n_estimators': n_estimators,
'max_features': max_features,
'max_depth': max_depth,
'min_samples_split': min_samples_split,
'min_samples_leaf': min_samples_leaf,
'bootstrap': bootstrap}

random_grid

{'n_estimators': [5, 10, 20, 40, 50, 80, 100],
 'max_features': ['auto', 'sqrt'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120],
 'min_samples_split': [2, 4, 6, 8, 10],
 'min_samples_leaf': [1, 2, 3, 4],
 'bootstrap': [True, False]}

In [38]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=3)

In [39]:
from sklearn.model_selection import RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator=rf,
    param_distributions=random_grid,
    n_iter=50,   #random search 탐색 횟
    cv=10,  # 교차 검증 fold 개수
    verbose=2,   #진행상황 (1:계산 시간 보여주기, 2: score도 보여주기)
    random_state=3,  #random.seed
    n_jobs=-1   , scoring = 'neg_mean_squared_error')    #선정기준 : -MSE

In [40]:
rf_random.fit(X_matrix, Y_vector)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
240 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/util

RandomizedSearchCV(cv=10, estimator=RandomForestRegressor(random_state=3),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      120],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 3, 4],
                                        'min_samples_split': [2, 4, 6, 8, 10],
                                        'n_estimators': [5, 10, 20, 40, 50, 80,
                                                         100]},
                   random_state=3, scoring='neg_mean_squared_error', verbose=2)

In [41]:
rf_random.best_params_   #Best Parameters

{'n_estimators': 40,
 'min_samples_split': 10,
 'min_samples_leaf': 3,
 'max_features': 'sqrt',
 'max_depth': 120,
 'bootstrap': True}

In [42]:
rf_full = RandomForestRegressor(**rf_random.best_params_, random_state = 3)
rf_full.fit(X_matrix, Y_vector)
rf_full

RandomForestRegressor(max_depth=120, max_features='sqrt', min_samples_leaf=3,
                      min_samples_split=10, n_estimators=40, random_state=3)

In [43]:
predict_rf = rf_full.predict(X_test)

In [44]:
##성능 평가
from sklearn.metrics import *
RMSE = root_mean_squared_error(Y_test, predict_rf)
MAE = mean_absolute_error(Y_test, predict_rf)
MAPE = mean_absolute_percentage_error(Y_test, predict_rf)

print(f'RMSE: {RMSE}', f'MAE: {MAE}', f'MAPE: {MAPE}')

RMSE: 8777.046757462838 MAE: 6581.3171401964655 MAPE: 0.2115408584448432


## 전체 데이터셋

In [45]:
!pip install gradio scikit-learn

In [46]:
import gradio as gr
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

In [47]:
feature_importance = rf_full.feature_importances_
features = X_test.columns

In [48]:
전체 = pd.read_csv("/content/drive/MyDrive/빅콘_다시/데이터/전체데이터/[완성]전체데이터.csv")
전체.head()

,EMD_CD,EMD_KOR_NM,시도명,시군구명,gfa_dlt,land_area,floor_gr,floor_bm,year_con,교통_도보접근시간,...,상권업종,M2평균,증가율평균,dealt_yr,dealt_qr,bld_type,의료,성비,총인구,GRDP
0,11110101,청운동,서울특별시,종로구,4518.11,624.5,13,3,1989,11.672584,...,3,4062.6,0.2,2024,4Q,일반,298391378,0.482768,35719537,139417
1,11110102,신교동,서울특별시,종로구,4518.11,624.5,13,3,1989,9.517857,...,1,4062.6,0.2,2024,4Q,일반,298391378,0.482768,35719537,139417
2,11110103,궁정동,서울특별시,종로구,4518.11,624.5,13,3,1989,7.641667,...,1,4062.6,0.2,2024,4Q,일반,298391378,0.482768,35719537,139417
3,11110104,효자동,서울특별시,종로구,4518.11,624.5,13,3,1989,6.497826,...,1,4062.6,0.2,2024,4Q,일반,298391378,0.482768,35719537,139417
4,11110105,창성동,서울특별시,종로구,4518.11,624.5,13,3,1989,4.734104,...,5,4062.6,0.2,2024,4Q,일반,298391378,0.482768,35719537,139417


In [49]:
np.array(전체["EMD_KOR_NM"])

array(['청운동', '신교동', '궁정동', '효자동', '창성동', '통의동', '적선동', '통인동', '누상동',
       '누하동', '옥인동', '체부동', '필운동', '내자동', '사직동', '도렴동', '당주동', '내수동',
       '세종로', '신문로1가', '신문로2가', '청진동', '서린동', '수송동', '중학동', '종로1가', '공평동',
       '관훈동', '견지동', '와룡동', '권농동', '운니동', '익선동', '경운동', '관철동', '인사동',
       '낙원동', '종로2가', '팔판동', '삼청동', '안국동', '소격동', '화동', '사간동', '송현동',
       '가회동', '재동', '계동', '원서동', '훈정동', '묘동', '봉익동', '돈의동', '장사동', '관수동',
       '종로3가', '인의동', '예지동', '원남동', '연지동', '종로4가', '효제동', '종로5가', '종로6가',
       '이화동', '연건동', '충신동', '동숭동', '혜화동', '명륜1가', '명륜2가', '명륜4가', '명륜3가',
       '창신동', '숭인동', '교남동', '평동', '송월동', '홍파동', '교북동', '행촌동', '구기동',
       '평창동', '부암동', '홍지동', '신영동', '무악동', '무교동', '다동', '태평로1가', '을지로1가',
       '을지로2가', '남대문로1가', '삼각동', '수하동', '장교동', '수표동', '소공동', '남창동', '북창동',
       '태평로2가', '남대문로2가', '남대문로3가', '남대문로4가', '남대문로5가', '봉래동1가', '봉래동2가',
       '회현동1가', '회현동2가', '회현동3가', '충무로1가', '충무로2가', '명동1가', '명동2가',
       '남산동1가', '남산동2가', '남산동3가', '저동1가', '충무로4가', '충무로5가', '인현동

In [50]:
전체.to_csv("/content/sample_data/전체.csv")

In [51]:
X_전체 = 전체[X_matrix.columns]
X_전체.head()

,gfa_dlt,land_area,floor_gr,floor_bm,year_con,교통_도보접근시간,학교_차량접근시간,교통_차량접근시간,학교_대중교통접근시간,병원_대중교통접근시간,...,상권업종,M2평균,증가율평균,dealt_yr,dealt_qr,bld_type,의료,성비,총인구,GRDP
0,4518.11,624.5,13,3,1989,11.672584,0.424488,22.998422,9.644970,14.516765,...,3,4062.6,0.2,2024,4Q,일반,298391378,0.482768,35719537,139417
1,4518.11,624.5,13,3,1989,9.517857,0.710142,22.584762,10.669643,13.553571,...,1,4062.6,0.2,2024,4Q,일반,298391378,0.482768,35719537,139417
2,4518.11,624.5,13,3,1989,7.641667,0.600417,21.353333,9.500000,12.916667,...,1,4062.6,0.2,2024,4Q,일반,298391378,0.482768,35719537,139417
3,4518.11,624.5,13,3,1989,6.497826,0.714130,20.747826,9.130435,12.782609,...,1,4062.6,0.2,2024,4Q,일반,298391378,0.482768,35719537,139417
4,4518.11,624.5,13,3,1989,4.734104,1.236994,20.489210,10.000000,11.136802,...,5,4062.6,0.2,2024,4Q,일반,298391378,0.482768,35719537,139417


In [52]:
X_전체.to_csv("/content/sample_data/X_전체.csv")

In [53]:
import gradio as gr
import math

index_mapping = {str(add_dong): idx for idx, add_dong in enumerate(sorted(전체['EMD_KOR_NM']))}

# 예측 함수 정의
def predict_rf_function(법정동,

                        #건물특징
                        gfa_dlt, year_con, land_area, floor_gr, floor_bm,

                        #경제지수
                        avg_부동산시장_소비심리지수, 차주가중종합지수_attitude, 대기업_attitude,
                        상호저축은행_attitude, 신용카드회사_attitude, 생명보험회사_attitude,
                        차주가중종합지수_risk, 대기업_risk, 상호저축은행_risk, 신용카드회사_risk,
                        차주가중종합지수_demand, 대기업_demand, 상호저축은행_demand, 신용카드회사_demand,
                        상호금융조합_demand, 생명보험회사_demand, avg_경제심리지수,
                        M2평균, 증가율평균):

    index = index_mapping[법정동]

    input_data = X_전체.iloc[[index]]
    input_data['dealt_qr'] = le.fit_transform(input_data['dealt_qr'])
    input_data['dealt_yr'] = le.fit_transform(input_data['dealt_yr'])
    input_data['bld_type'] = le.fit_transform(input_data['bld_type'])

    #건물특징
    input_data['gfa_dlt'] = gfa_dlt
    input_data['year_con'] = year_con
    input_data['land_area'] = land_area
    input_data['floor_gr'] = floor_gr
    input_data['floor_bm'] = floor_bm
    #input_data['dealt_yr'] = le.fit_transform(dealt_yr)  #범주형
    #input_data['dealt_qr'] = le.fit_transform(dealt_qr)  #범주형
    #input_data['bld_type'] = le.fit_transform(bld_type)   #범주형

    #경제지수
    input_data['avg_부동산시장_소비심리지수'] = avg_부동산시장_소비심리지수
    input_data['차주가중종합지수_attitude'] = 차주가중종합지수_attitude
    input_data['대기업_attitude'] = 대기업_attitude
    input_data['상호저축은행_attitude'] = 상호저축은행_attitude
    input_data['신용카드회사_attitude'] = 신용카드회사_attitude
    input_data['생명보험회사_attitude'] = 생명보험회사_attitude
    input_data['차주가중종합지수_risk'] = 차주가중종합지수_risk
    input_data['대기업_risk'] = 대기업_risk
    input_data['상호저축은행_risk'] = 상호저축은행_risk
    input_data['신용카드회사_risk'] = 신용카드회사_risk
    input_data['차주가중종합지수_demand'] = 차주가중종합지수_demand
    input_data['대기업_demand'] = 대기업_demand
    input_data['상호저축은행_demand'] = 상호저축은행_demand
    input_data['신용카드회사_demand'] = 신용카드회사_demand
    input_data['상호금융조합_demand'] = 상호금융조합_demand
    input_data['생명보험회사_demand'] = 생명보험회사_demand
    input_data['avg_경제심리지수'] = avg_경제심리지수
    input_data['M2평균'] = M2평균
    input_data['증가율평균'] = 증가율평균


    # 예측값 도출
    Y_pred = rf_full.predict(input_data)
    Y_pred = format(math.trunc(Y_pred[0] * 1000), ",")
    #lowerCI = format(math.trunc((Y_pred - 1.96*se) * 1000), ",")
    #upperCI = format(math.trunc((Y_pred + 1.96*se) * 1000), ",")
    #return f"'{법정동}'오피스 당거래가 예측값(3.3m2당거래가):" f"'{lowerCI}'" f" ~ " f"'{upperCI}'"
    return f"'{법정동}'오피스 당거래가 예측값(3.3m2당거래가):'{Y_pred}'"
# Gradio 인터페이스 정의
demo = gr.Interface(
    fn=predict_rf_function,
     inputs=[
        gr.Dropdown(choices=list(sorted(전체['EMD_KOR_NM'])), label="법정동"),

        #건물특징
        gr.Number(label="m2 기준 연면적(gfa_dlt)"),
        gr.Number(label="준공년도(year_con)"),
        gr.Number(label="m2 기준 토지면적(land_area)"),
        gr.Number(label="지상층(floor_gr)"),
        gr.Number(label="지하층(floor_bm)"),
        #gr.Number(label="거래년도(dealt_yr)", value = 2024),
        #gr.Dropdown(choices=list(["1Q", "2Q", "3Q", "4Q"]), label = "거래분기(dealt_qr)", value = "4Q"),
        #gr.Dropdown(choices=list(["집합","일반"]), label = "건물유형(bld_type)"),

        #경제지수
        gr.Number(label="avg_부동산시장_소비심리지수", value =X_전체["avg_부동산시장_소비심리지수"].unique()[0]),
        gr.Number(label="차주가중종합지수_attitude", value = X_전체["차주가중종합지수_attitude"].unique()),
        gr.Number(label="대기업_attitude", value = X_전체["대기업_attitude"].unique()),
        gr.Number(label="상호저축은행_attitude",  value = X_전체["상호저축은행_attitude"].unique()),
        gr.Number(label="신용카드회사_attitude", value = X_전체["신용카드회사_attitude"].unique()),
        gr.Number(label="생명보험회사_attitude", value = X_전체["생명보험회사_attitude"].unique()),
        gr.Number(label="차주가중종합지수_risk", value = X_전체["차주가중종합지수_risk"].unique()),
        gr.Number(label="대기업_risk", value = X_전체["대기업_risk"].unique()),
        gr.Number(label="상호저축은행_risk", value = X_전체["상호저축은행_risk"].unique()),
        gr.Number(label="신용카드회사_risk", value = X_전체["신용카드회사_risk"].unique()),
        gr.Number(label="차주가중종합지수_demand", value = X_전체["차주가중종합지수_demand"].unique()),
        gr.Number(label='대기업_demand', value = X_전체["대기업_demand"].unique()),
        gr.Number(label="상호저축은행_demand", value = X_전체["상호저축은행_demand"].unique()),
        gr.Number(label="신용카드회사_demand", value = X_전체["신용카드회사_demand"].unique()),
        gr.Number(label="상호금융조합_demand", value = X_전체["상호금융조합_demand"].unique()),
        gr.Number(label="생명보험회사_demand", value = X_전체["생명보험회사_demand"].unique()),
        gr.Number(label="avg_경제심리지수", value = X_전체["avg_경제심리지수"].unique()),
        gr.Number(label="M2평균", value = X_전체["M2평균"].unique()),
        gr.Number(label="증가율평균", value = X_전체["증가율평균"].unique())
    ],
    outputs="text",
    title="오피스 매매 당거래가 예측값 using Random Forest",
    description="[빅콘2024, 부동산 주제] 팀명: 통계핑(이민주)"
)

# 앱 실행
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5b89a3ab980716c18c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [54]:
with open("app.py", "w") as f:
    f.write('''

    import gradio as gr
import math

index_mapping = {str(add_dong): idx for idx, add_dong in enumerate(sorted(전체['EMD_KOR_NM']))}

# 예측 함수 정의
def predict_rf_function(법정동,

                        #건물특징
                        gfa_dlt, year_con, land_area, floor_gr, floor_bm,

                        #경제지수
                        avg_부동산시장_소비심리지수, 차주가중종합지수_attitude, 대기업_attitude,
                        상호저축은행_attitude, 신용카드회사_attitude, 생명보험회사_attitude,
                        차주가중종합지수_risk, 대기업_risk, 상호저축은행_risk, 신용카드회사_risk,
                        차주가중종합지수_demand, 대기업_demand, 상호저축은행_demand, 신용카드회사_demand,
                        상호금융조합_demand, 생명보험회사_demand, avg_경제심리지수,
                        M2평균, 증가율평균):

    index = index_mapping[법정동]

    input_data = X_전체.iloc[[index]]
    input_data['dealt_qr'] = le.fit_transform(input_data['dealt_qr'])
    input_data['dealt_yr'] = le.fit_transform(input_data['dealt_yr'])
    input_data['bld_type'] = le.fit_transform(input_data['bld_type'])

    #건물특징
    input_data['gfa_dlt'] = gfa_dlt
    input_data['year_con'] = year_con
    input_data['land_area'] = land_area
    input_data['floor_gr'] = floor_gr
    input_data['floor_bm'] = floor_bm
    #input_data['dealt_yr'] = le.fit_transform(dealt_yr)  #범주형
    #input_data['dealt_qr'] = le.fit_transform(dealt_qr)  #범주형
    #input_data['bld_type'] = le.fit_transform(bld_type)   #범주형

    #경제지수
    input_data['avg_부동산시장_소비심리지수'] = avg_부동산시장_소비심리지수
    input_data['차주가중종합지수_attitude'] = 차주가중종합지수_attitude
    input_data['대기업_attitude'] = 대기업_attitude
    input_data['상호저축은행_attitude'] = 상호저축은행_attitude
    input_data['신용카드회사_attitude'] = 신용카드회사_attitude
    input_data['생명보험회사_attitude'] = 생명보험회사_attitude
    input_data['차주가중종합지수_risk'] = 차주가중종합지수_risk
    input_data['대기업_risk'] = 대기업_risk
    input_data['상호저축은행_risk'] = 상호저축은행_risk
    input_data['신용카드회사_risk'] = 신용카드회사_risk
    input_data['차주가중종합지수_demand'] = 차주가중종합지수_demand
    input_data['대기업_demand'] = 대기업_demand
    input_data['상호저축은행_demand'] = 상호저축은행_demand
    input_data['신용카드회사_demand'] = 신용카드회사_demand
    input_data['상호금융조합_demand'] = 상호금융조합_demand
    input_data['생명보험회사_demand'] = 생명보험회사_demand
    input_data['avg_경제심리지수'] = avg_경제심리지수
    input_data['M2평균'] = M2평균
    input_data['증가율평균'] = 증가율평균


    # 예측값 도출
    Y_pred = rf_full.predict(input_data)
    Y_pred = format(math.trunc(Y_pred[0] * 1000), ",")
    #lowerCI = format(math.trunc((Y_pred - 1.96*se) * 1000), ",")
    #upperCI = format(math.trunc((Y_pred + 1.96*se) * 1000), ",")
    #return f"'{법정동}'오피스 당거래가 예측값(3.3m2당거래가):" f"'{lowerCI}'" f" ~ " f"'{upperCI}'"
    return f"'{법정동}'오피스 당거래가 예측값(3.3m2당거래가):'{Y_pred}'"
# Gradio 인터페이스 정의
demo = gr.Interface(
    fn=predict_rf_function,
     inputs=[
        gr.Dropdown(choices=list(sorted(전체['EMD_KOR_NM'])), label="법정동"),

        #건물특징
        gr.Number(label="m2 기준 연면적(gfa_dlt)"),
        gr.Number(label="준공년도(year_con)"),
        gr.Number(label="m2 기준 토지면적(land_area)"),
        gr.Number(label="지상층(floor_gr)"),
        gr.Number(label="지하층(floor_bm)"),
        #gr.Number(label="거래년도(dealt_yr)", value = 2024),
        #gr.Dropdown(choices=list(["1Q", "2Q", "3Q", "4Q"]), label = "거래분기(dealt_qr)", value = "4Q"),
        #gr.Dropdown(choices=list(["집합","일반"]), label = "건물유형(bld_type)"),

        #경제지수
        gr.Number(label="avg_부동산시장_소비심리지수", value =X_전체["avg_부동산시장_소비심리지수"].unique()[0]),
        gr.Number(label="차주가중종합지수_attitude", value = X_전체["차주가중종합지수_attitude"].unique()),
        gr.Number(label="대기업_attitude", value = X_전체["대기업_attitude"].unique()),
        gr.Number(label="상호저축은행_attitude",  value = X_전체["상호저축은행_attitude"].unique()),
        gr.Number(label="신용카드회사_attitude", value = X_전체["신용카드회사_attitude"].unique()),
        gr.Number(label="생명보험회사_attitude", value = X_전체["생명보험회사_attitude"].unique()),
        gr.Number(label="차주가중종합지수_risk", value = X_전체["차주가중종합지수_risk"].unique()),
        gr.Number(label="대기업_risk", value = X_전체["대기업_risk"].unique()),
        gr.Number(label="상호저축은행_risk", value = X_전체["상호저축은행_risk"].unique()),
        gr.Number(label="신용카드회사_risk", value = X_전체["신용카드회사_risk"].unique()),
        gr.Number(label="차주가중종합지수_demand", value = X_전체["차주가중종합지수_demand"].unique()),
        gr.Number(label='대기업_demand', value = X_전체["대기업_demand"].unique()),
        gr.Number(label="상호저축은행_demand", value = X_전체["상호저축은행_demand"].unique()),
        gr.Number(label="신용카드회사_demand", value = X_전체["신용카드회사_demand"].unique()),
        gr.Number(label="상호금융조합_demand", value = X_전체["상호금융조합_demand"].unique()),
        gr.Number(label="생명보험회사_demand", value = X_전체["생명보험회사_demand"].unique()),
        gr.Number(label="avg_경제심리지수", value = X_전체["avg_경제심리지수"].unique()),
        gr.Number(label="M2평균", value = X_전체["M2평균"].unique()),
        gr.Number(label="증가율평균", value = X_전체["증가율평균"].unique())
    ],
    outputs="text",
    title="오피스 매매 당거래가 예측값 using Random Forest",
    description="[빅콘2024, 부동산 주제] 팀명: 통계핑(이민주)"
)

# 앱 실행
demo.launch(share=True)''')


In [55]:
전체["EMD_KOR_NM"].unique().shape[0]

483